In [5]:
!pip install numpy manim


  Using cached manim-0.18.1-py3-none-any.whl.metadata (10 kB)
  Using cached cloup-3.0.5-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached isosurfaces-0.1.2-py3-none-any.whl.metadata (3.3 kB)
  Using cached ManimPango-0.6.0-cp312-cp312-win_amd64.whl.metadata (8.2 kB)
  Using cached mapbox_earcut-1.0.2-cp312-cp312-win_amd64.whl.metadata (2.2 kB)
  Using cached moderngl-5.12.0-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached moderngl_window-2.4.6-py3-none-any.whl.metadata (8.8 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached pyrr-0.10.3-py3-none-any.whl.metadata (832 bytes)
Using cached manim-0.18.1-py3-none-any.whl (585 kB)
Using cached cloup-3.0.5-py2.py3-none-any.whl (54 kB)
Using cached isosurfaces-0.1.2-py3-none-any.whl (11 kB)
Using cached ManimPango-0.6.0-cp312-cp312-win_amd64.whl (4.1 MB)
Using cached mapbox_earcut-1.0.2-cp312-cp312-win_amd64.whl (73 kB)
Using cached moderngl-5.12.0-cp312-cp312-win_amd64.whl (108 kB)
Using ca

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
torchvision 0.19.1 requires torch==2.4.1, but you have torch 2.3.1 which is incompatible.


In [ ]:
import numpy as np
import random
from manim import *

# Create synthetic data
np.random.seed(0)
X_train = np.linspace(0, 10, 100)
y_train = 3 * X_train + np.random.randn(100) * 2

class GDRegressor:
    def __init__(self, learning_rate=0.01, epochs=100):
        self.intercept_ = 0
        self.coef_ = None
        self.lr = learning_rate
        self.epochs = epochs

    def fit(self, X_train, y_train):
        self.coef_ = np.zeros(X_train.shape[1])
        for _ in range(self.epochs):
            y_pred = (X_train @ self.coef_) + self.intercept_
            intercept_gradient = -2 * np.mean(y_train - y_pred)
            coef_gradient = -2 * (X_train.T @ (y_train - y_pred)) / X_train.shape[0]
            self.intercept_ -= self.lr * intercept_gradient
            self.coef_ -= self.lr * coef_gradient

    def predict(self, X_test):
        return (X_test @ self.coef_) + self.intercept_

class MBGDRegressor:
    def __init__(self, learning_rate=0.01, n_iter=10, batch=10):
        self.coef_ = None
        self.intercept_ = 0
        self.learning_rate = learning_rate
        self.n_iterations = n_iter
        self.batch = batch

    def fit(self, X_train, y_train):
        n_samples, n_features = X_train.shape
        self.coef_ = np.zeros(n_features)

        for _ in range(self.n_iterations):
            for _ in range(int(n_samples / self.batch)):
                idx = random.sample(range(n_samples), self.batch)
                y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ -= self.learning_rate * intercept_der
                coef_der = -2 * np.dot((y_train[idx] - y_hat), X_train[idx])
                self.coef_ -= self.learning_rate * coef_der

    def predict(self, X_test):
        return (X_test @ self.coef_) + self.intercept_

class SDGRegressor:
    def __init__(self, learning_rate=0.01, n_iter=100):
        self.learning_rate = learning_rate
        self.n_iterations = n_iter
        self.coef_ = None
        self.intercept_ = 0

    def fit(self, X_train, y_train):
        n_samples, n_features = X_train.shape
        self.coef_ = np.zeros(n_features)

        for _ in range(self.n_iterations):
            for idx in range(n_samples):
                idx = np.random.randint(0, n_samples)
                y_pred = np.dot(X_train[idx], self.coef_) + self.intercept_
                intercept_gradient = -2 * (y_train[idx] - y_pred)
                coef_gradient = -2 * (y_train[idx] - y_pred) * X_train[idx]
                self.intercept_ -= self.learning_rate * intercept_gradient
                self.coef_ -= self.learning_rate * coef_gradient

    def predict(self, X_test):
        return (X_test @ self.coef_) + self.intercept_

class GradientDescentScene(Scene):
    def construct(self):
        # Prepare data for fitting
        X_train_reshaped = X_train.reshape(-1, 1)  # Reshape for the models

        # Initialize and fit models
        gd_model = GDRegressor(learning_rate=0.01, epochs=100)
        gd_model.fit(X_train_reshaped, y_train)

        mbgd_model = MBGDRegressor(learning_rate=0.01, n_iter=10, batch=10)
        mbgd_model.fit(X_train_reshaped, y_train)

        sdg_model = SDGRegressor(learning_rate=0.01, n_iter=100)
        sdg_model.fit(X_train_reshaped, y_train)

        # Create axes
        axes = Axes(
            x_range=[0, 10, 1],
            y_range=[-10, 40, 5],
            axis_config={"color": BLUE},
        )

        # Create dots for training data
        dots = [Dot(point=axes.c2p(x, y), color=WHITE) for x, y in zip(X_train, y_train)]
        self.add(axes, *dots)

        # Create prediction lines
        gd_line = axes.plot(lambda x: gd_model.predict(np.array([[x]])), color=YELLOW, x_range=[0, 10])
        mbgd_line = axes.plot(lambda x: mbgd_model.predict(np.array([[x]])), color=GREEN, x_range=[0, 10])
        sdg_line = axes.plot(lambda x: sdg_model.predict(np.array([[x]])), color=RED, x_range=[0, 10])

        # Add lines to the scene
        self.play(Create(gd_line), run_time=2, label='Batch Gradient Descent')
        self.wait(1)
        self.play(Transform(gd_line, mbgd_line), run_time=2, label='Mini-Batch Gradient Descent')
        self.wait(1)
        self.play(Transform(mbgd_line, sdg_line), run_time=2, label='Stochastic Gradient Descent')
        self.wait(2)

# To run the scene, use the following command in your terminal:
# manim -pql <filename>.py GradientDescentScene
